# Case Study - Apriori

Head of Marketing xyz.com meminta Anda untuk mengimplementasikan mesin rekomendasi berdasarkan analisis market basket yang telah Anda pelajari sebelumnya.

Jika sebelumnya Anda hanya bermain-main dengan data yang ukurannya kecil, kini saatnya Anda bekerja dengan data yang sesungguhnya.
Anda memutuskan untuk membangun model Aprioari dari transaksi 1 bulan terakhir.
Setelah mengeksplorasi data di database transaction, Anda meng-export data tersebut ke dalam sebuah text file dengan nama `market_basket.txt`.

Data yang akan Anda proses ini memiliki dimensi 29475 x 10340.
Artinya, ada lebih dari 29 ribu transaksi dengan total 10 ribu lebih barang.
Masing - masing baris pada file menandakan satu transaksi.
Sedangkan angka yang tertulis menunjukkan ID barang.

Contoh:
```
29,27,28
30,31,32
45,44,35,36
45,3,36
```

- Transaksi ke-1, barang yang dibeli adalah id barang 29, 27, dan 28.
- Transaksi ke-2, barang yang dibeli adalah id barang 30, 31, dan 32.
- dst.

Karena dimensi datanya yang cukup besar, Anda tidak bisa membuat matrix ukuran mxn (m = jumlah transaksi, n=jumlah barang) karena akan menghabiskan memory komputer Anda.
Namun Anda tahu bahwa matrix tersebut akan memiliki nilai mayoritas 0.
Maka Anda bisa menggunakan sparse matrix.

Untuk membuat sparse matrix Anda harus menggunakan library Scipy.
Untuk lebih jelasnya bisa dibaca di laman berikut [https://docs.scipy.org/doc/scipy/reference/sparse.html](https://docs.scipy.org/doc/scipy/reference/sparse.html)

In [87]:
import numpy as np
from scipy.sparse import csc_matrix 


rows = []
columns = []
data = []

# baca data dari text file
with open("market_basket.txt", 'r') as f:
    for i, line in enumerate(f):
        line = line.strip()
        
        for x in np.unique(line.split(',')):
            rows.append(i)
            columns.append(int(x))
            data.append(1)

data_sparse = csc_matrix((data, (rows, columns)), shape=(max(rows) + 1, max(columns) + 1), dtype=np.uint8)

Selanjutnya Anda tinggal mengikuti langkah - langkah yang sama dengan yang pernah Anda pelajari sebelumnya.


In [82]:
from collections import defaultdict, Counter
from itertools import combinations


# convert sparse matrix ke numpy array
data = data_sparse.toarray()


# definisikan minimum support
minimum_support = 500


###############################
# inisialisasi variabel
###############################

# frequent_itemsets berupa dictionary
# key: jumlah item di itemset 1, 2, ... dst
# value: set yang berisi itemset
# itemset : tuple(id_barang1, id_barang2, dst)
# kenapa set? karena kita harus memastikan kumpulan itemset yang unik
frequent_itemsets = defaultdict(set)


# support berupa defaultdict yang valuenya bertipe integer
# menggunakan built-in class Counter
# Counter == defaultdict(int)
support = Counter()


###############################
# langkah 1
###############################

# 1.1. hitung frekuensi itemset - itemset dengan jumlah item = 1
frequency = data.sum(axis=0)

# 1.2. cek jika frekuensi tiap itemset >= minimum support
# maka masukkan ke dalam variabel frequent_itemsets
for item, count in enumerate(frequency):
    if count >= minimum_support:
        support[(item,)] = count
        frequent_itemsets[1].add((item,))
        
# display(support)
# display(frequent_itemsets)
        
        
###############################
# langkah 2
# hitung frekuensi itemset dengan jumlah item = 2, 3, dst
###############################

k = 2
while frequent_itemsets[k - 1]:
    # 2.1. cari kandidat itemset dengan jumlah item k
    ids = []
    for itemset in frequent_itemsets[k - 1]:
        for itemid in itemset:
            ids.append(itemid)
    ids = list(set(ids))
    itemsets = combinations(ids, k)
    
    # tidak efisien
    # itemsets = combinations(range(10310), k)

    # 2.2. hitung frekensi masing2 untuk masing - masing kandidat itemset yang muncul di data transaksi
    for itemset in itemsets:
        
        # 2.2.1. hitung frekuensi
        frequency = np.argwhere(data[:, itemset].sum(axis=1) == len(itemset)).shape[0]
        
        # 2.2.2. cek apakah kandidat itemset memenuhi minimum support
        if frequency >= minimum_support:
            support[itemset] += frequency
            frequent_itemsets[k].add(itemset)

    k += 1

display(support)
display(frequent_itemsets)

Counter({(29,): 3644,
         (33,): 587,
         (35,): 3322,
         (36,): 13226,
         (38,): 2065,
         (45,): 8024,
         (107,): 853,
         (165,): 628,
         (33, 35): 511,
         (35, 36): 1387,
         (35, 165): 586,
         (35, 107): 812,
         (35, 45): 656,
         (36, 38): 1374,
         (36, 45): 4610,
         (36, 29): 1113,
         (38, 45): 603,
         (45, 29): 856})

defaultdict(set,
            {1: {(29,), (33,), (35,), (36,), (38,), (45,), (107,), (165,)},
             2: {(33, 35),
              (35, 36),
              (35, 45),
              (35, 107),
              (35, 165),
              (36, 29),
              (36, 38),
              (36, 45),
              (38, 45),
              (45, 29)},
             3: set()})

In [92]:
minimum_confidence = 0.6
rules = {}
k = 2
while frequent_itemsets[k]:
    
    # untuk setiap itemset dengan jumlah item k
    for itemset in frequent_itemsets[k]:
        
        # generate subset
        # dengan jumlah item 1, 2, dst
        for i in range(1, len(itemset)):
            subsets = combinations(itemset, i)
            
            # di tiap subset
            for subset in subsets:
                
                # cek apakah probability lebih besar daripada minimum confidence
                if support[itemset] / support[subset] >= minimum_confidence:
                    rules[subset] = set(itemset) - set(subset)
            
    k += 1
    
display(rules)

{(33,): {35}, (38,): {36}, (107,): {35}, (165,): {35}}

## Quiz

1. Tampilkan beberapa sampel input-output rekomendasi
2. Bagaimana menurut Anda kualitas rekomendasi yang dihasilkan? 
3. Apa yang mempengaruhi hasil rekomendasi model Anda tersebut?
4. Sebutkan beberapa kelemahan dari algoritma Apriori
5. Bagaimana strategi Anda untuk untuk memperbaiki kekurangan - kekurangan tersebut
